In [ ]:
from functools import partial
from typing import List, Tuple

import numpy as np
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import HTML, display, update_display

In [ ]:
class Colors:
    START = 'green'
    GOAL = 'blue'
    OBSTACLE = 'black'
    PATH = 'red'
    VISITED = 'yellow'


class Interface:
    def __init__(self, shape=(10,10)):
        self.n_rows, self.n_cols = shape
        self.buttons = [
            [
                widgets.Button(
                    description='',
                    layout=Layout(width='50px', height='50px')
                )
                for _ in range(self.n_cols)
            ]
            for _ in range(self.n_rows)
        ]

        self.map_widget = VBox([HBox(row) for row in self.buttons])
        self.link_positions()

        self.start_pos, self.goal_pos = None, None
        self.obstacles = set()

    def on_select_position(self, pos: tuple, button: widgets.Button):
        """Callback on user click on a position of the map

        :param pos: row, column of button on grid
        :param button: clicked button
        """
        if self.start_pos is None:
            self.start_pos = pos
            button.style.button_color = Colors.START

        elif self.goal_pos is None:
            self.goal_pos = pos
            button.style.button_color = Colors.GOAL

        else:
            self.obstacles.add(pos)
            button.style.button_color = Colors.OBSTACLE

    def link_positions(self):
        """Link clicks on buttons"""
        for i, row in enumerate(self.buttons):
            for j, button in enumerate(row):
                button.on_click(partial(self.on_select_position, (i, j)))

    def disable_buttons(self):
        """Disable buttons to avoid clicks after end game"""
        for row in self.buttons:
            for button in row:
                button.disabled = True

    def start(self):
        """Display map interface"""
        display(self.map_widget)



In [ ]:
interface = Interface()
interface.start()

In [ ]:
MOVES_BFS = [(+1, 0), (-1, 0), (0, +1), (0, -1)]


def BFS(start_pos: Tuple[int, int], goal_pos: Tuple[int, int],
        obstacles: List[Tuple[int, int]], max_y: int, max_x: int):
    """Breadth-first search algorithm"""

    def is_valid(position: Tuple[int, int]):
        """Checks if a position is valid
        i.e. it is inside map and it is not a obstacle"""
        y, x = position
        return (x >= 0 and x < max_x and
                y >= 0 and y < max_y and
                position not in obstacles)

    def get_next_positions(y, x):
        """Get next possible moves"""
        return [
            new_pos for dy, dx in MOVES_BFS
            if is_valid(new_pos := (y + dy, x + dx))
        ]

    visited_positions = set()
    to_visit = [(start_pos, [])]  # position, path_to_position

    while to_visit:
        pos, path = to_visit.pop(0)

        if pos == goal_pos:
            return path, visited_positions

        if pos in visited_positions:
            continue

        visited_positions.add(pos)

        for next_pos in get_next_positions(*pos):
            to_visit.append((next_pos, [*path, pos]))


    return [], visited_positions

# A*
Parte 1

Utilizar notebook base da aula passada (BFS)

Parte 2

Implementar regra para obter movimentações possíveis (cima, baixo, direita e esquerda, diagonal) com custos diferente para diagonal (1.4x custo cima baixo)

Implementar função que remove movimentos proibidos (fora do mapa ou sobre obstáculos)

Parte 3   - Implementar busca heurística A*


Receba o mapa
1. Inicialize a lista aberta

2. Inicialize a lista fechada

coloque o nó inicial aberto

  lista (você pode deixar seu f em zero)

3. até que a lista aberta esteja vazia

  a) encontre o nó com o menor f em

     a lista aberta, chame-a de "q"

  b) retirar q da lista aberta



  c) gerar os 8 sucessores de q e definir seus

     pais para q

   

  d) para cada sucessor

      i) se o sucessor for o objetivo, pare a busca

     

      ii) caso contrário, calcule g e h para o sucessor

        sucessor.g = q.g + distância entre

                            sucessor e q

        sucessor.h = distância do gol até

        sucessor (Isso pode ser feito usando muitos

        maneiras, discutiremos três heurísticas-

        Manhattan, Diagonal e Euclidiana

        Heurística)

       

        sucessor.f = sucessor.g + sucessor.h

      iii) se um nó com a mesma posição que

          sucessor está na lista OPEN que tem um

         menor f que o sucessor, pule este sucessor

      iV) se um nó com a mesma posição que

          sucessor está na lista FECHADA que tem

          um f menor que o sucessor, pule este sucessor

          caso contrário, adicione o nó à lista aberta

   fim (para loop)



  e) pressione q na lista fechada

  fim (loop while)

#  <font color="red">Seu código vem aqui</font>

In [ ]:

MOVES = [((+1, 0), 1), ((-1, 0), 1), ((0, +1), 1), ((0, -1), 1),
         ((+1, +1), 1.4), ((-1, +1), 1.4), ((+1, -1), 1.4), ((-1, -1), 1.4)]
         # quais são os movimentos possíveis (adicione diagonais)

def a_star(start_pos: Tuple[int, int], goal_pos: Tuple[int, int],
        obstacles: List[Tuple[int, int]], max_y: int, max_x: int):
    """Breadth-first search algorithm"""

    def heuristics(position: Tuple[int, int]):
       return abs(position[0] - goal_pos[0]) + abs(position[1] - goal_pos[1])

    def is_valid(position: Tuple[int, int]):
        """Checks if a position is valid
        i.e. it is inside map and it is not a obstacle"""
        y, x = position
        return (x >= 0 and x < max_x and
                y >= 0 and y < max_y and
                position not in obstacles)

    def get_next_positions(y, x):
        """Get next possible moves"""
        return [
            (new_pos, cost) for (dy, dx), cost in MOVES
            if is_valid(new_pos := (y + dy, x + dx))
        ]


    g = {start_pos: 0}
    f = {start_pos: 0 + heuristics(start_pos)}
    path = {start_pos: []}

    visited_positions = set()
    to_visit = [start_pos]  # position, path_to_position

    while to_visit:
        pos = min(to_visit, key=lambda pos: f[pos])
        to_visit.remove(pos)

        if pos == goal_pos:
            return path[pos], visited_positions

        if pos in visited_positions:
            continue

        visited_positions.add(pos)

        for next_pos, cost in get_next_positions(*pos):
            new_g = g[pos] + cost
            if next_pos not in g or new_g < g[next_pos]:
                g[next_pos] = new_g
                f[next_pos] = new_g + heuristics(next_pos)
                path[next_pos] = [*path[pos], pos] # Correção da ordem do caminho
                if next_pos not in to_visit:
                    to_visit.append(next_pos)

    return [], visited_positions

In [ ]:
class PathFinder(Interface):
    def __init__(self, shape=(10, 10)):
        super().__init__(shape)
        self.find_button = widgets.Button(
            description='Find shortest path A*',
            style=widgets.ButtonStyle(button_color=Colors.START)
        )

        self.find_button_bfs = widgets.Button(
            description='BFS',
            style=widgets.ButtonStyle(button_color=Colors.START)
        )

        self.clear_button = widgets.Button(
            description='clear',
            style=widgets.ButtonStyle(button_color=Colors.VISITED)
        )

        self.find_button.on_click(self.search)
        self.find_button_bfs.on_click(self.search_bfs)
        self.clear_button.on_click(self.clear)

    def clear(self, *args):
        for row in self.buttons:
            for cell in row:
                if cell.style.button_color in [Colors.VISITED, Colors.PATH]:
                    cell.style.button_color = None

    def set_path(self, path: List[Tuple[int, int]]):
        """Color a path in map"""
        for y, x in path[1:]:
            self.buttons[y][x].style.button_color = Colors.PATH

    def set_visited(self, visited: List[Tuple[int, int]]):
        """Color a path in map"""
        for y, x in visited:
            if self.buttons[y][x].style.button_color not in {Colors.START, Colors.GOAL, Colors.OBSTACLE, Colors.PATH}:
                self.buttons[y][x].style.button_color = Colors.VISITED

    def search(self, *args):
        """Disable editions on map, then finds the shorterst path and draw it"""
        self.disable_buttons()
        path, visited_positions = a_star(self.start_pos, self.goal_pos, self.obstacles, self.n_rows,
                      self.n_cols)
        print(len(visited_positions))
        self.set_visited(visited_positions)
        self.set_path(path)
        return path

    def search_bfs(self, *args):
        """Disable editions on map, then finds the shorterst path and draw it"""
        self.disable_buttons()
        path, visited_positions = BFS(self.start_pos, self.goal_pos, self.obstacles, self.n_rows,
                      self.n_cols)
        print(len(visited_positions))
        self.set_visited(visited_positions)
        self.set_path(path)
        return path

    def start(self):
        """Display map interface"""
        display(VBox([self.map_widget, self.find_button,
                      self.find_button_bfs, self.clear_button]))



path_finder = PathFinder()
path_finder.start()

63
14
14
14
63
14


Respostas:
1. O A*

2. O algoritimo BFS ou "Busca cega" faz a analize de campos definidos antes de avançar para o próximo, o que consome mais recurso. O A* ele somente faz a analize quando possui obstaculos, que ajuda na tomada de decisão. quando o caminho é livre, ele consegue seguir direto.

3. Defini a A*, Tem melhor eficiência e praticidade do que a busca cega.

4. Consegue, Ela pega as posições de onde ela está e de onde ela consegue ir, que foram definidos na função MOVES, em seguida ele faz a previsão com base de onde ele quer chegar, ele faz os movimentos e quando encontra obstaculos ele se parece muito com a BFS, ele vê os obstáculos e chega no objetivo final.

# <font color="red">Responda</font>

1. Qual algorítimo visita menos posições?
2. O que torna o algorítimo mais eficiente?
3. Qual função heuristica você definiu? Por quê?
4. Sua função heurística consegue acertar o custo futuro? Por que ela é aceita?